# RAGAS Evaluation - Measuring RAG Quality

This notebook evaluates our FlexiRAG system using RAGAS metrics:
- **Faithfulness**: Is the LLM hallucinating?
- **Answer Relevancy**: Does the answer match the question?
- **Context Precision**: Are retrieved chunks relevant?
- **Context Recall**: Did we get all necessary information?

In [ ]:
import sys
sys.path.append('..')

from src.vector_store import initialize_chroma_db
from src.qa_chain import ask_question
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)
import pandas as pd

/Users/mirac/Desktop/RAGDocumentationAssistant/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 1: Test Dataset

Creating test questions for both domains with ground truth answers

In [ ]:
# Automotive Domain Test Cases
automotive_test_cases = [
    {
        "question": "What is CAN protocol used for?",
        "ground_truth": "CAN (Controller Area Network) is a vehicle bus standard designed to allow microcontrollers and devices to communicate with each other without a host computer.",
        "domain": "automotive"
    },
    {
        "question": "What does OBD-II stand for?",
        "ground_truth": "OBD-II stands for On-Board Diagnostics II, which is a standardized system for vehicle self-diagnostics and reporting.",
        "domain": "automotive"
    },
    {
        "question": "What is the main advantage of CAN FD over CAN?",
        "ground_truth": "CAN FD (CAN with Flexible Data-Rate) enables higher data rates up to 8 Mbit/s and allows data payloads up to 64 bytes, compared to CAN's 1 Mbit/s and 8 bytes.",
        "domain": "automotive"
    }
]

# Fashion Domain Test Cases
fashion_test_cases = [
    {
        "question": "What types of women's clothing are available?",
        "ground_truth": "The fashion dataset includes various categories such as Westernwear, Indianwear, Lingerie & Nightwear, Footwear, Watches, Jewellery, and Fragrance for women.",
        "domain": "fashion"
    },
    {
        "question": "Are there products under 1000 rupees?",
        "ground_truth": "Yes, the fashion dataset contains many products with selling prices under 1000 rupees across various categories.",
        "domain": "fashion"
    }
]

# Combine all test cases
all_test_cases = automotive_test_cases + fashion_test_cases
print(f"Total test cases: {len(all_test_cases)}")

Total test cases: 5


## Step 2: Initialize System & Get Answers

In [ ]:
# Initialize ChromaDB
print("Loading ChromaDB...")
client, collection = initialize_chroma_db()
print(f"✅ Loaded {collection.count()} documents")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


Loading ChromaDB...
Initializing ChromaDB at: ./chroma_db
✅ Loaded existing collection: documents
   Documents in collection: 0
✅ Loaded 0 documents


In [ ]:
# Get answers from our RAG system
results = []

for i, test_case in enumerate(all_test_cases, 1):
    print(f"\nTest {i}/{len(all_test_cases)}: {test_case['question'][:50]}...")
    
    # Query with domain filtering
    filter_metadata = {"domain": test_case['domain']}
    result = ask_question(
        collection, 
        test_case['question'],
        n_results=3,
        filter_metadata=filter_metadata
    )
    
    # Prepare data for RAGAS
    results.append({
        "question": test_case['question'],
        "answer": result['answer'],
        "contexts": result['retrieved_chunks'],
        "ground_truth": test_case['ground_truth']
    })
    
    print(f"✅ Answer generated")

print(f"\n✅ Collected {len(results)} results")


Test 1/5: What is CAN protocol used for?...

Question: What is CAN protocol used for?
Step 1: Retrieving relevant chunks...
Loading embedding model: all-MiniLM-L6-v2...


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


✅ Model loaded!
✅ Retrieved 0 chunks

Step 2: Generating answer with LLM...


## Step 3: Prepare Dataset for RAGAS

In [ ]:
# Convert to HuggingFace Dataset format
dataset_dict = {
    "question": [r["question"] for r in results],
    "answer": [r["answer"] for r in results],
    "contexts": [r["contexts"] for r in results],
    "ground_truth": [r["ground_truth"] for r in results]
}

dataset = Dataset.from_dict(dataset_dict)
print(f"Dataset created with {len(dataset)} examples")
print(f"\nDataset columns: {dataset.column_names}")

Dataset created with 5 examples

Dataset columns: ['question', 'answer', 'contexts', 'ground_truth']


## Step 4: Run RAGAS Evaluation

**Note:** This uses Ollama locally for LLM-based metrics.
Make sure Ollama is running with llama3.2 model.

In [ ]:
# Configure RAGAS to use Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings

# Use our local Ollama instance
llm = ChatOllama(model="llama3.2:3b")
embeddings = OllamaEmbeddings(model="llama3.2:3b")

print("✅ Configured RAGAS to use local Ollama")

✅ Configured RAGAS to use local Ollama


/var/folders/8y/n0smdvfd3qz_4tdt6tf78vmc0000gn/T/ipykernel_12062/253370650.py:6: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.2:3b")
/var/folders/8y/n0smdvfd3qz_4tdt6tf78vmc0000gn/T/ipykernel_12062/253370650.py:7: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama3.2:3b")


In [ ]:
# Run evaluation
print("Running RAGAS evaluation...")
print("This may take a few minutes...\n")

try:
    result = evaluate(
        dataset,
        metrics=[
            faithfulness,
            answer_relevancy,
            context_precision,
            context_recall
        ],
        llm=llm,
        embeddings=embeddings
    )
    
    print("\n✅ Evaluation complete!")
except Exception as e:
    print(f"⚠️  Note: Some RAGAS metrics require OpenAI API.")
    print(f"Error: {e}")
    print("\nRunning with available metrics only...")
    
    # Fallback: Run only metrics that work with Ollama
    result = evaluate(
        dataset,
        metrics=[answer_relevancy],  # This one works with local LLM
        llm=llm,
        embeddings=embeddings
    )

Running RAGAS evaluation...
This may take a few minutes...



Evaluating:  50%|█████     | 10/20 [01:21<02:09, 12.93s/it]

## Step 5: Analyze Results

In [ ]:
# Convert to DataFrame for analysis
df = result.to_pandas()

print("\n" + "="*60)
print("RAGAS Evaluation Results")
print("="*60)

# Show overall scores
print("\n📊 Overall Scores:")
print("-"*60)
for metric in df.columns:
    if metric not in ['question', 'answer', 'contexts', 'ground_truth']:
        score = df[metric].mean()
        print(f"{metric.capitalize()}: {score:.3f}")

# Show detailed results
print("\n📝 Detailed Results:")
print("-"*60)
display(df)

## Step 6: Interpret Results

**Score Interpretation:**
- **0.0 - 0.4**: Poor - Needs significant improvement
- **0.4 - 0.6**: Fair - Acceptable but can be better
- **0.6 - 0.8**: Good - Performing well
- **0.8 - 1.0**: Excellent - High quality RAG system

**Metrics Explained:**
- **Faithfulness**: Measures hallucination. Higher = less hallucination.
- **Answer Relevancy**: Measures if answer matches question. Higher = more relevant.
- **Context Precision**: Measures if retrieved chunks are relevant. Higher = better retrieval.
- **Context Recall**: Measures if all needed info was retrieved. Higher = nothing missed.

In [ ]:
# Save results
df.to_csv('../evaluation_results.csv', index=False)
print("\n✅ Results saved to: evaluation_results.csv")

## Summary

This evaluation provides quantitative metrics for our FlexiRAG system quality.
Use these scores to:
1. Compare different RAG configurations
2. Track improvements over time
3. Identify weak points (low scores)
4. Demonstrate system quality in portfolio